In [1]:
import os
import numpy as np
import pandas as pd
import swifter
from sklearn.model_selection import train_test_split
import utils
import catboost

In [2]:
DATA_PATH = "./data"

In [3]:
test = utils.load_full_test_csv(DATA_PATH)

In [ ]:
closest_hits_features = test.swifter.apply(
    utils.find_closest_hit_per_station, result_type="expand", axis=1)
test_concat = pd.concat(
    [test.loc[:, utils.SIMPLE_FEATURE_COLUMNS], closest_hits_features],
    axis=1)

Pandas Apply:  44%|████▍     | 321139/726095 [04:33<05:20, 1262.18it/s]

In [ ]:
model = catboost.CatBoostClassifier().load_model("track_2_model.cbm")
predictions = model.predict(test_concat.values, prediction_type="RawFormulaVal").astype(np.float32)
prediction_pd = pd.DataFrame(data={"prediction": predictions}, index=test.index)
prediction_pd.to_csv("track_2_submission_python.csv", index_label="id")